In [4]:
import pandas as pd
import numpy as np
from prophet import Prophet
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# 1. Đọc dữ liệu cổ phiếu cùng ngành (Ngân hàng làm ví dụ)
stocks = {
    'MBB': 'MBB.csv',
    'BID': 'Dữ liệu Lịch sử BID 245.4T.csv',
    'VCB': 'Dữ liệu Lịch sử VCB 481.29T.csv',
    'CTG': 'Dữ liệu Lịch sử CTG 201.91T.csv',
    'TCB': 'Dữ liệu Lịch sử TCB 190.04T.csv'
}
capitalization = {'MBB': 141.88, 'BID': 245.4, 'VCB': 481.29, 'CTG': 201.91, 'TCB': 190.04}  # Đơn vị: nghìn tỷ

data_dict = {}
for stock, file in stocks.items():
    df = pd.read_csv(file)
    df['Ngày'] = pd.to_datetime(df['Ngày'], format='%d/%m/%Y')
    df = df[(df['Ngày'] >= pd.to_datetime('2020-04-10')) & (df['Ngày'] < pd.to_datetime('2025-04-10'))]
    df['Lần cuối'] = df['Lần cuối'].astype(str).str.replace(',', '').astype(float)
    df = df.rename(columns={'Ngày': 'Date', 'Lần cuối': f'{stock}_Price'})
    data_dict[stock] = df[['Date', f'{stock}_Price']]

# Ghép dữ liệu
industry_data = data_dict['MBB'].copy()
for stock in ['BID', 'VCB', 'CTG', 'TCB']:
    industry_data = industry_data.merge(data_dict[stock], on='Date', how='left')
industry_data = industry_data.ffill()

# Tính chỉ số ngành (trung bình có trọng số theo vốn hóa)
total_cap = sum(capitalization.values())
industry_data['Banking_Industry_Index'] = 0
for stock in stocks.keys():
    industry_data['Banking_Industry_Index'] += industry_data[f'{stock}_Price'] * (capitalization[stock] / total_cap)
industry_data = industry_data[['Date', 'Banking_Industry_Index']]

# Lưu dữ liệu lịch sử chỉ số ngành
industry_data.to_csv('Banking_Industry_Historical.csv', index=False)
print("Dữ liệu lịch sử chỉ số ngành ngân hàng đã được lưu vào 'Banking_Industry_Historical.csv'")

# 2. Dự đoán chỉ số ngành bằng Prophet
model_prophet = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
model_prophet.fit(industry_data.rename(columns={'Date': 'ds', 'Banking_Industry_Index': 'y'}))

# Tạo khung thời gian tương lai (từ 10/4/2025 đến 1/5/2028)
future_dates = pd.date_range(start='2025-04-10', end='2028-05-01', freq='D')
future_df = pd.DataFrame({'ds': future_dates})
forecast = model_prophet.predict(future_df)
industry_forecast = forecast[['ds', 'yhat']]
industry_forecast.columns = ['Date', 'Banking_Industry_Index']

# Lưu dự đoán đầu tháng
industry_forecast['Date'] = pd.to_datetime(industry_forecast['Date'])
industry_monthly = industry_forecast.resample('MS', on='Date').first().reset_index()  # Giữ cột 'Date'
industry_monthly.to_csv('Banking_Industry_Monthly_Forecast.csv', index=False)
print("Dự đoán đầu tháng chỉ số ngành ngân hàng đã được lưu vào 'Banking_Industry_Monthly_Forecast.csv'")

# 3. Dự đoán giá ngày bằng LSTM
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(industry_data[['Banking_Industry_Index']])

time_step = 300
def create_lstm_data(data, time_step):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

X, y = create_lstm_data(scaled_data, time_step)

X_train_tensor = torch.FloatTensor(X).unsqueeze(-1)  # Thêm chiều cho LSTM: (samples, time_step, 1)
y_train_tensor = torch.FloatTensor(y).reshape(-1, 1)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=2, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

model_lstm = LSTMModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_lstm.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_lstm = model_lstm.to(device)

num_epochs = 30
for epoch in range(num_epochs):
    model_lstm.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model_lstm(X_batch)
        loss = criterion(outputs, y_batch)
        optimizer.zero_grad()
        torch.nn.utils.clip_grad_norm_(model_lstm.parameters(), max_norm=1.0)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}')

# 4. Dự đoán giá ngày trong tương lai
historical_data = scaled_data.flatten().tolist()  # Đảm bảo là danh sách các giá trị số
future_dates_daily = pd.date_range(start='2025-04-10', end='2028-05-01', freq='D')
daily_df = pd.DataFrame({'Date': future_dates_daily})
scaled_future_data = np.zeros((len(future_dates_daily), 1))

model_lstm.eval()
predictions_daily = []
current_window = historical_data[-time_step:]  # Lấy window cuối cùng từ dữ liệu lịch sử

for i in range(len(future_dates_daily)):
    # Tạo tensor 3D: (1, time_step, 1)
    current_window_tensor = torch.FloatTensor([current_window[-time_step:]]).unsqueeze(-1).to(device)  # (1, time_step, 1)
    with torch.no_grad():
        pred = model_lstm(current_window_tensor).cpu().numpy()[0, 0]
        predictions_daily.append(pred)
        current_window.append(pred)  # Thêm giá trị dự đoán trực tiếp

# Đảo ngược chuẩn hóa
predictions_daily = np.array(predictions_daily).reshape(-1, 1)
predictions_rescaled = scaler.inverse_transform(predictions_daily)

# Kết hợp dự đoán ngày với dự đoán tháng (scale)
industry_predictions = pd.DataFrame({'Date': future_dates_daily, 'Banking_Industry_Index': predictions_rescaled.flatten()})
industry_predictions['Date'] = pd.to_datetime(industry_predictions['Date'])
industry_monthly_merged = industry_monthly.merge(industry_predictions, on='Date', how='left')
industry_predictions['Banking_Industry_Index'] = industry_monthly_merged['Banking_Industry_Index_y'].fillna(
    industry_monthly_merged['Banking_Industry_Index_x']
)

# Lưu kết quả
industry_predictions.to_csv('Banking_Industry_Daily_Predictions.csv', index=False)
print("Dự đoán ngày chỉ số ngành ngân hàng đã được lưu vào 'Banking_Industry_Daily_Predictions.csv'")

# 5. Biểu đồ minh họa
plt.figure(figsize=(14, 7))
plt.plot(industry_data['Date'], industry_data['Banking_Industry_Index'], label='Lịch sử', color='blue')
plt.plot(industry_predictions['Date'], industry_predictions['Banking_Industry_Index'], label='Dự đoán', color='green', linestyle='--')
plt.title('Dự đoán Chỉ Số Ngành Ngân Hàng (2020-2028)')
plt.xlabel('Ngày')
plt.ylabel('Chỉ Số Ngành')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('Banking_Industry_Prediction_Chart.png')
plt.close()
print("Biểu đồ dự đoán chỉ số ngành đã được lưu vào 'Banking_Industry_Prediction_Chart.png'")

18:09:55 - cmdstanpy - INFO - Chain [1] start processing


Dữ liệu lịch sử chỉ số ngành ngân hàng đã được lưu vào 'Banking_Industry_Historical.csv'


18:09:55 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\Tuan Anh\AppData\Local\Temp\ipykernel_21392\1034335422.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  industry_forecast['Date'] = pd.to_datetime(industry_forecast['Date'])


Dự đoán đầu tháng chỉ số ngành ngân hàng đã được lưu vào 'Banking_Industry_Monthly_Forecast.csv'
Epoch [1/30], Loss: 0.021307
Epoch [2/30], Loss: 0.024577
Epoch [3/30], Loss: 0.008915
Epoch [4/30], Loss: 0.003754
Epoch [5/30], Loss: 0.003651
Epoch [6/30], Loss: 0.002038
Epoch [7/30], Loss: 0.004893
Epoch [8/30], Loss: 0.002487
Epoch [9/30], Loss: 0.001449
Epoch [10/30], Loss: 0.002346
Epoch [11/30], Loss: 0.002254
Epoch [12/30], Loss: 0.002437
Epoch [13/30], Loss: 0.002187
Epoch [14/30], Loss: 0.002939
Epoch [15/30], Loss: 0.001935
Epoch [16/30], Loss: 0.001302
Epoch [17/30], Loss: 0.001557
Epoch [18/30], Loss: 0.003363
Epoch [19/30], Loss: 0.001307
Epoch [20/30], Loss: 0.002643
Epoch [21/30], Loss: 0.001463
Epoch [22/30], Loss: 0.001400
Epoch [23/30], Loss: 0.001407
Epoch [24/30], Loss: 0.002149
Epoch [25/30], Loss: 0.000385
Epoch [26/30], Loss: 0.000866
Epoch [27/30], Loss: 0.001202
Epoch [28/30], Loss: 0.002032
Epoch [29/30], Loss: 0.000878
Epoch [30/30], Loss: 0.003920
Dự đoán ngày